In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from tqdm import tqdm
%matplotlib inline
import xarray

In [10]:
batch_path = '/mnt/c/Users/nozom/Desktop/mass-suite/example_data/batch/'

In [89]:
reference = pd.read_csv(batch_path + os.listdir(batch_path)[0], sep="\\t", engine='python')
reference

,PeakID,Title,Scans,RT left(min),RT (min),RT right (min),Precursor m/z,Height,Area,Model masses,...,m/z matched,MS/MS matched,RT similarity,Dot product,Reverse dot product,Fragment presence %,Total score,S/N,MS1 isotopes,MSMS spectrum
0,0,Unknown,98,0.539500,0.578250,0.628050,179.0358,7111.606934,26772.347656,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,37.786630,179.03441:8407 179.15318:232 180.02725:430 180...,NaN
1,1,Unknown,99,0.522900,0.583783,0.644650,201.0726,2514.433594,9090.625000,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,12.048300,201.01082:1272 201.07156:3386 202.06113:796 20...,NaN
2,2,Unknown,99,0.539500,0.583783,0.644650,217.0489,5024.579102,17694.775391,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,13.964120,216.94678:204 217.0489:5932 217.09781:1981 218...,NaN
3,3,Unknown,100,0.539500,0.589317,0.683383,142.0350,4195.795898,17614.511719,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,16.470440,141.96747:2344 142.03386:5165 142.12091:251 14...,NaN
4,4,Unknown,100,0.550583,0.589317,0.650183,158.0119,17884.183594,64892.605469,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,28.776020,158.01195:21778 158.12296:580 158.99753:8504 1...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1178,1178,Unknown,3818,21.036350,21.163620,21.241100,204.9978,1982.336914,17016.837891,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,21.838220,204.99425:2008 205.08414:341 206.94487:509 206...,NaN
1179,1179,Unknown,3819,20.931220,21.169150,21.313030,167.9657,1975.691650,19431.066406,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,11.131220,167.96436:2101 168.12469:285 168.96909:216 169...,NaN
1180,1180,Unknown,3822,21.141480,21.185750,21.335170,163.9683,5076.373535,39115.789062,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,49.431030,163.96829:5638 164.13103:216 164.97917:344 165...,NaN
1181,1181,Unknown,3826,21.069550,21.207900,21.246630,158.1247,1131.811279,3277.271240,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,8.522174,157.97536:14564 158.01187:41655 158.12381:1152...,NaN


In [90]:
# thresholds
peak_area = 50000
rt_window = 1.5
s_n = 5

sample_df = pd.DataFrame()
# create empty dataframe
num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))])
for f in range(num_files-1):
    standard = pd.read_csv(batch_path + os.listdir(batch_path)[f+1], sep="\\t", engine='python')
    # was gonna append each row from the standard only if they passed the filter thresholds
    for i in range(len(standard)):
        if ((standard.at[i,'Area']) > (peak_area and standard.iat[i,4] < rt_window) & (standard.at[i,'S/N'] > s_n)):
            sample_df = sample_df.append(standard.iloc[i])

In [91]:
sample_df
# This got messed up for some reason lol, was envisioning itll look similar to the reference but with more rows

,Adduct,Annotation tag (VS1.0),Area,Comment,Dot product,Formula,Fragment presence %,Height,InChIKey,Isotope,...,RT similarity,Reference RT,Reference m/z,Reverse dot product,S/N,SMILES,Scans,Title,Total score,m/z matched
0,[M+H]+,999.0,8615.794922,NaN,NaN,NaN,NaN,2381.034180,NaN,M + 0,...,NaN,NaN,NaN,NaN,13.050340,NaN,98.0,Unknown,NaN,0.0
1,[M+H]+,999.0,16394.250000,NaN,NaN,NaN,NaN,4384.651367,NaN,M + 0,...,NaN,NaN,NaN,NaN,27.317790,NaN,98.0,Unknown,NaN,0.0
2,[M+H]+,999.0,19556.232422,NaN,NaN,NaN,NaN,5029.392090,NaN,M + 0,...,NaN,NaN,NaN,NaN,16.679890,NaN,98.0,Unknown,NaN,0.0
3,[M+H]+,999.0,17208.224609,similar chromatogram in higher mz_0,NaN,NaN,NaN,4574.145508,NaN,M + 0,...,NaN,NaN,NaN,NaN,19.419960,NaN,99.0,Unknown,NaN,0.0
4,[M+H]+,999.0,82808.906250,NaN,NaN,NaN,NaN,18665.806641,NaN,M + 0,...,NaN,NaN,NaN,NaN,27.890480,NaN,99.0,Unknown,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1178,[M+H]+,999.0,18278.093750,NaN,NaN,NaN,NaN,2275.715332,NaN,M + 0,...,NaN,NaN,NaN,NaN,52.373000,NaN,3812.0,Unknown,NaN,0.0
1179,[M+H]+,999.0,29594.367188,NaN,NaN,NaN,NaN,2810.108887,NaN,M + 0,...,NaN,NaN,NaN,NaN,25.339680,NaN,3813.0,Unknown,NaN,0.0
1180,[M+H]+,999.0,13022.894531,NaN,NaN,NaN,NaN,1785.752441,NaN,M + 0,...,NaN,NaN,NaN,NaN,15.779640,NaN,3817.0,Unknown,NaN,0.0
1181,[M+H]+,999.0,82636.648438,NaN,NaN,NaN,NaN,10666.778320,NaN,M + 0,...,NaN,NaN,NaN,NaN,21.389860,NaN,3823.0,Unknown,NaN,0.0


In [88]:
# error
rt_error = 0.1
mz_error =0.00002

for i in range(len(sample_df)):
    for j in range(len(reference)):
        if((abs(sample_df[i,4]-reference[j,4]) < rt_error) and (abs(sample_df[i,6]-reference[j,6]) < mz_error)):
            reference = reference.append(sample_df.iloc[i])                                     

KeyError: (0, 4)